## 1.载入数据集

In [39]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np
import logging
import sys


def load_data():
    x, y = load_iris(return_X_y=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                        test_size=0.3, random_state=20)
    return x_train, x_test, y_train, y_test

## 2.定义节点

In [26]:
class Node(object):
    """
    定义KD树中的节点信息
    """

    def __init__(self, data=None, index=-1):
        self.data = data
        self.left_child = None
        self.right_child = None
        self.index = index

    def __str__(self):
        """
        打印节点信息
        :return:
        """
        return f"data({self.data}),index({int(self.index)})"

node = Node(np.array([1,2]), 0)
print(node)

data([1 2]),index(0)


## 3.定义LP距离

In [5]:
def distance(p1, p2, p=2):
    """

    :param p1:
    :param p2:
    :param p:  当p=2时为欧式距离
    :return:
    """
    return np.sum((p1 - p2) ** p) ** (1 / p)

## 4.定义KD树

In [31]:
class MyKDTree(object):
    """
    定义MyKDTree
    Parameters:
        points: 构造KD树用到的样本点，必须为np.array()类型，形状为 [n,m]
        p: p=2时为欧式距离
    """

    def __init__(self, points, p=2):
        self.root = None
        self.p = p
        self.dim = points.shape[1]
        self.distance = distance
        # 在原始样本的最后一列加入一列索引，表示每个样本点的序号
        points = np.hstack(([points, np.arange(0, len(points)).reshape(-1, 1)]))
        self.insert(points, order=0)  # 递归构建KD树

    def is_empty(self):
        return not self.root

    def level_order(self):
        """
        层次遍历
        :return:
        """
        root = self.root
        if not root:
            return []
        queue = [root]
        res = []
        while queue:
            tmp = []
            for i in range(len(queue)):
                node = queue.pop(0)
                tmp.append(node)
                if node.left_child:
                    queue.append(node.left_child)
                if node.right_child:
                    queue.append(node.right_child)
            res.append(tmp)
        print("\n层次遍历结果为：")
        for i, r in enumerate(res):
            print(f"第{i + 1}层的节点为：")
            for node in r:
                print(f"<p({node.data}), idx({int(node.index)})>")
            print("\n")
            
    def insert(self, data, order=0):
        """
        在KD树中插入节点
        :param data: 必须为np.array()类型，形状为 [n,m]
        :param order: 用来判断比较的维度
        :return:
        """
        if len(data) < 1:
            return
        data = sorted(data, key=lambda x: x[order % self.dim])  # 按某个维度进行排序
        print(f"当前待划分样本点：{[item.tolist() for item in data]}")
        idx = len(data) // 2
        node = Node(data[idx][:-1], data[idx][-1])
        print(f"父节点：{data[idx]}")
        left_data = data[:idx]
        print(f"左子树: {[item.tolist() for item in left_data]}")
        right_data = data[idx + 1:]
        print(f"右子树: {[item.tolist() for item in right_data]}")
        print("============")
        if self.is_empty():
            self.root = node  # 整个KD树的根节点
        node.left_child = self.insert(left_data, order + 1)  # 递归构建左子树
        node.right_child = self.insert(right_data, order + 1)  # 递归构建右子树
        return node

    def nearest_search(self, point):
        """
        最近邻搜索
        :param point: 必须为np.array()类型
        :return:
        """
        best_node = None
        best_dist = np.inf
        visited = []  # 用来记录哪些节点被访问过
        point = point.reshape(-1)

        def nearest_node_search(point, curr_node, order=0):
            """
            :param point:  shape: (n,)
            :param curr_node: data: shape (n,)
            :param order:
            :return:
            """
            nonlocal best_node, best_dist, visited  # 声明这三个变量不是局部变量
            print(f"当前访问节点为：{curr_node}")
            if curr_node is None:
                print(f"结束本次递归")
                return None
            visited.append(curr_node)
            dist = self.distance(curr_node.data, point, self.p)
            print(f"当前访问节点到被搜索点的距离为：{round(dist, 3)}")
            print(f"【上次】:全局最佳距离为：{round(best_dist, 3)}, 全局最佳点为：{best_node}")
            if dist < best_dist:
                best_dist = dist
                best_node = curr_node
            print(f"【本次】:全局最佳距离为：{round(best_dist, 3)}, 全局最佳点为：{best_node}\n")
            cmp_dim = order % self.dim
            if point[cmp_dim] < curr_node.data[cmp_dim]:
                print(f"访问当前节点{curr_node}的左孩子")
                nearest_node_search(point, curr_node.left_child, order + 1)
            else:
                print(f"访问当前节点{curr_node}的右孩子")
                nearest_node_search(point, curr_node.right_child, order + 1)
            print(f"回到上一层递归，当前访问节点为: {curr_node}，开始判断步骤(6)")
            if np.abs(curr_node.data[cmp_dim] - point[cmp_dim]) < best_dist:
                print(f"** {curr_node.data}满足条件被搜索点到当前节点划分维度的距离小于全局最短距离，即 "
                             f"|{curr_node.data[cmp_dim]} - {point[cmp_dim]}| < {round(best_dist, 3)} **")
                child = curr_node.left_child if curr_node.left_child not in visited else curr_node.right_child
                tmp = '左' if curr_node.left_child not in visited else '右'
                print(f"访问当前节点{curr_node}的{tmp}孩子")
                nearest_node_search(point, child, order + 1)

        nearest_node_search(point, self.root, 0)
        return best_node, best_dist

    def append(self, k_nearest_nodes, curr_node, point):
        """
        将当前节点加入到k_nearest_nodes中并按每个节点到被搜索点的距离升序排列
        :param k_nearest_nodes: list，用来保存到目前位置距离被搜索点最近的K个点
        :param curr_node:  Node()类型，当前访问的节点
        :param point: 被搜索点，np.array()类型，形状为(m,)
        :return:
        """
        k_nearest_nodes.append(curr_node)
        k_nearest_nodes = sorted(k_nearest_nodes,
                                 key=lambda x: self.distance(x.data, point, self.p))
        print(f"\t\t当前K近邻有序列表中的节点为（已按距离升序排序）：")
        for item in k_nearest_nodes:
            print(f"\t\t\t{item}", )
        print("\n")
        return k_nearest_nodes


    def k_nearest_search(self, points, k):
        """
        分别查找points中每个样本距其最近的k个样本点
        :param points: np.array()   形状为[n,m]
        :param k:
        :return:
        """
        result_points = []
        result_ind = []
        for point in points:
            k_nodes = self._k_nearest_search(point, k)
            tmp_points = []
            tmp_ind = []
            for node in k_nodes:
                tmp_points.append(node.data)
                tmp_ind.append(int(node.index))
            result_points.append(tmp_points)
            result_ind.append(tmp_ind)
        return np.array(result_points), np.array(result_ind)

    def _k_nearest_search(self, point, k):
        """
        寻找距离样本点point最近的k个样本点
        :param point: np.array()   形状为[m,]
        :param k:
        :return:
        """
        k_nearest_nodes = []
        visited = []
        n = 0
        print(f"\n\n=========== 正在查找离样本点{point}最近的{k}个样本点！==========\n")

        def k_nearest_node_search(point, curr_node, order=0):
            nonlocal k_nearest_nodes, n
            print(f"K近邻搜索当前访问节点为：{curr_node}")
            if curr_node is None:
                return None
            visited.append(curr_node)
            if n < k:  # 如果当前还没找到k个点，则直接进行保存
                print(f"\t有序列表中的节点数目为：{n} < {k}，直接加入新节点并排序")
                n += 1
                k_nearest_nodes = self.append(k_nearest_nodes, curr_node, point)
            else:  # 已经找到k个局部最优点，开始进行筛选
                d1 = self.distance(curr_node.data, point, self.p)
                d2 = self.distance(point, k_nearest_nodes[-1].data, self.p)
                print(f"\t被搜索点{point}到当前节点{curr_node}的距离为 {round(d1, 3)}")
                print(f"\t被搜索点{point}到列表中最后一个节点{k_nearest_nodes[-1]}的距离为 {round(d2, 3)}")
                if d1 < d2:
                    print(
                        f"\t当前节点到被搜索点的距离{round(d1, 3)} < 被搜索点到列表中最后一个节点的距离{round(d2, 3)}，进行替换")
                    k_nearest_nodes.pop()  # 移除最后一个
                    k_nearest_nodes = self.append(k_nearest_nodes, curr_node, point)  # 加入新的点并进行排序
            cmp_dim = order % self.dim
            if point[cmp_dim] < curr_node.data[cmp_dim]:
                print(f"访问当前节点{curr_node}的左孩子")
                k_nearest_node_search(point, curr_node.left_child, order + 1)
            else:
                print(f"访问当前节点{curr_node}的右孩子")
                k_nearest_node_search(point, curr_node.right_child, order + 1)
            print(f"回到上一层递归，当前访问节点为 {curr_node}，开始判断步骤(6)")
            if n < k or np.abs(curr_node.data[cmp_dim] - point[cmp_dim]) < \
                    self.distance(point, k_nearest_nodes[-1].data, self.p):
                print(
                    f"** 被搜索点到当前节点划分维度的距离小于列表中最后一个元素到被搜索点的距离，即 "
                    f"|{curr_node.data[cmp_dim]} - {point[cmp_dim]}| < "
                    f"{round(self.distance(point, k_nearest_nodes[-1].data, self.p), 3)} **")
                child = curr_node.left_child if curr_node.left_child not in visited else curr_node.right_child
                tmp = '左' if curr_node.left_child not in visited else '右'
                print(f"访问当前节点{curr_node}的{tmp}孩子")
                k_nearest_node_search(point, child, order + 1)

        k_nearest_node_search(point, self.root, 0)
        return k_nearest_nodes

## 5 运行结果

### 5.1 构建KD树

In [28]:
points = np.array(
        [[5, 7], [3, 8], [6, 3], [8, 5], [15, 6.], [10, 4], [12, 13], [9, 10], [11, 14]])

def test_kd_tree_build(points):
    print("MyKDTree 运行结果：")
    print("构建KD树")
    tree = MyKDTree(points)
    print("构建结束")
    print("MyKDTree 层次遍历结果：")
    tree.level_order()

test_kd_tree_build(points)

MyKDTree 运行结果：
构建KD树
当前待划分样本点：[[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [9.0, 10.0, 7.0], [10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
父节点：[ 9. 10.  7.]
左子树: [[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [5.0, 7.0, 0.0], [3.0, 8.0, 1.0]]
父节点：[5. 7. 0.]
左子树: [[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[3.0, 8.0, 1.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
父节点：[8. 5. 3.]
左子树: [[6.0, 3.0, 2.0]]
右子树: []
当前待划分样本点：[[6.0, 3.0, 2.0]]
父节点：[6. 3. 2.]
左子树: []
右子树: []
当前待划分样本点：[[3.0, 8.0, 1.0]]
父节点：[3. 8. 1.]
左子树: []
右子树: []
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0], [12.0, 13.0, 6.0], [11.0, 14.0, 8.0]]
父节点：[12. 13.  6.]
左子树: [[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
右子树: [[11.0, 14.0, 8.0]]
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
父节点：[15.  6.  4.]
左子树: [[10.0, 4.0, 5.0]]
右子树: []
当前待划分样本点：[

### 5.2 KD树最近邻搜索

In [32]:
def test_kd_nearest_search(points, q=None):
    tree = MyKDTree(points)
    best_node, best_dist = tree.nearest_search(q)
    print("MyKDTree 运行结果：")
    print(f"离样本点{q}最近的节点是：{best_node},距离为：{round(best_dist, 3)}")

    kd_tree = KDTree(points)
    dist, ind = kd_tree.query(q, k=1)
    print("sklearn KDTree 运行结果：")
    print(f"离样本点{q}最近的节点是：{points[ind]},距离为：{dist}")

test_kd_nearest_search(points, q=np.array([[8.9, 4]]))

当前待划分样本点：[[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [9.0, 10.0, 7.0], [10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
父节点：[ 9. 10.  7.]
左子树: [[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [5.0, 7.0, 0.0], [3.0, 8.0, 1.0]]
父节点：[5. 7. 0.]
左子树: [[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[3.0, 8.0, 1.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
父节点：[8. 5. 3.]
左子树: [[6.0, 3.0, 2.0]]
右子树: []
当前待划分样本点：[[6.0, 3.0, 2.0]]
父节点：[6. 3. 2.]
左子树: []
右子树: []
当前待划分样本点：[[3.0, 8.0, 1.0]]
父节点：[3. 8. 1.]
左子树: []
右子树: []
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0], [12.0, 13.0, 6.0], [11.0, 14.0, 8.0]]
父节点：[12. 13.  6.]
左子树: [[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
右子树: [[11.0, 14.0, 8.0]]
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
父节点：[15.  6.  4.]
左子树: [[10.0, 4.0, 5.0]]
右子树: []
当前待划分样本点：[[10.0, 4.0, 5.0]]
父节点

### 5.3 KD树K近邻搜索

In [33]:
def test_kd_k_nearest_search():
    points = np.array(
        [[5, 7], [3, 8], [6, 3], [8, 5], [15, 6.], [10, 4], [12, 13], [9, 10], [11, 14]])
    tree = MyKDTree(points)
    p = np.array([[8.9, 4]])
    k_best_nodes, ind = tree.k_nearest_search(p, k=3)
    print("MyKDTree 运行结果：")
    print(f"\n{k_best_nodes}")
    print(f"样本索引编号为:{ind}")

    print("sklearn KDTree 运行结果：")
    kd_tree = KDTree(points)
    dist, ind = kd_tree.query(p, k=3)
    print(f"\n{points[ind]}", )
    print(f"样本索引编号为:{ind}")
test_kd_k_nearest_search()

当前待划分样本点：[[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [9.0, 10.0, 7.0], [10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
父节点：[ 9. 10.  7.]
左子树: [[3.0, 8.0, 1.0], [5.0, 7.0, 0.0], [6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[10.0, 4.0, 5.0], [11.0, 14.0, 8.0], [12.0, 13.0, 6.0], [15.0, 6.0, 4.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0], [5.0, 7.0, 0.0], [3.0, 8.0, 1.0]]
父节点：[5. 7. 0.]
左子树: [[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
右子树: [[3.0, 8.0, 1.0]]
当前待划分样本点：[[6.0, 3.0, 2.0], [8.0, 5.0, 3.0]]
父节点：[8. 5. 3.]
左子树: [[6.0, 3.0, 2.0]]
右子树: []
当前待划分样本点：[[6.0, 3.0, 2.0]]
父节点：[6. 3. 2.]
左子树: []
右子树: []
当前待划分样本点：[[3.0, 8.0, 1.0]]
父节点：[3. 8. 1.]
左子树: []
右子树: []
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0], [12.0, 13.0, 6.0], [11.0, 14.0, 8.0]]
父节点：[12. 13.  6.]
左子树: [[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
右子树: [[11.0, 14.0, 8.0]]
当前待划分样本点：[[10.0, 4.0, 5.0], [15.0, 6.0, 4.0]]
父节点：[15.  6.  4.]
左子树: [[10.0, 4.0, 5.0]]
右子树: []
当前待划分样本点：[[10.0, 4.0, 5.0]]
父节点

## 6 KNN实现

In [34]:
class MyKNN():
    def __init__(self, n_neighbors, p=2):
        """
        :param n_neighbors:
        :param p: 当p=2时表示欧氏距离
        """
        self.n_neighbors = n_neighbors
        self.p = p

    def fit(self, x, y):
        self._y = y
        self.kd_tree = MyKDTree(x, self.p)
        return self

    @staticmethod
    def get_pred_labels(query_label):
        """
        根据query_label返回每个样本对应的标签
        :param query_label: 二维数组， query_label[i] 表示离第i个样本最近的k个样本点对应的正确标签
        :return:
        """
        y_pred = [0] * len(query_label)
        for i, label in enumerate(query_label):
            max_freq = 0
            count_dict = {}
            for l in label:
                count_dict[l] = count_dict.setdefault(l, 0) + 1
                if count_dict[l] > max_freq:
                    max_freq = count_dict[l]
                    y_pred[i] = l
        return np.array(y_pred)

    def predict(self, x):
        k_best_nodes, ind = self.kd_tree.k_nearest_search(x, k=self.n_neighbors)
        query_label = self._y[ind]
        y_pred = self.get_pred_labels(query_label)
        return y_pred

## 7 测试对于结果

In [37]:

x_train, x_test, y_train, y_test = load_data()
k = 5
my_model = MyKNN(n_neighbors=k)
my_model.fit(x_train, y_train)
y_pred = my_model.predict(x_test)
print(f"impl_by_ours 准确率：{accuracy_score(y_test, y_pred)}")

model = KNeighborsClassifier(n_neighbors=k)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f"impl_by_sklearn 准确率：{accuracy_score(y_test, y_pred)}")

当前待划分样本点：[[4.4, 3.0, 1.3, 0.2, 25.0], [4.4, 2.9, 1.4, 0.2, 45.0], [4.4, 3.2, 1.3, 0.2, 56.0], [4.5, 2.3, 1.3, 0.3, 12.0], [4.6, 3.1, 1.5, 0.2, 78.0], [4.6, 3.4, 1.4, 0.3, 91.0], [4.6, 3.6, 1.0, 0.2, 100.0], [4.7, 3.2, 1.3, 0.2, 0.0], [4.7, 3.2, 1.6, 0.2, 66.0], [4.8, 3.1, 1.6, 0.2, 33.0], [4.8, 3.4, 1.9, 0.2, 61.0], [4.8, 3.0, 1.4, 0.3, 65.0], [4.9, 3.1, 1.5, 0.1, 24.0], [5.0, 3.3, 1.4, 0.2, 2.0], [5.0, 2.0, 3.5, 1.0, 34.0], [5.0, 3.6, 1.4, 0.2, 48.0], [5.0, 2.3, 3.3, 1.0, 59.0], [5.0, 3.5, 1.6, 0.6, 74.0], [5.0, 3.0, 1.6, 0.2, 89.0], [5.0, 3.4, 1.5, 0.2, 92.0], [5.0, 3.4, 1.6, 0.4, 96.0], [5.0, 3.5, 1.3, 0.3, 97.0], [5.1, 3.8, 1.9, 0.4, 4.0], [5.1, 3.4, 1.5, 0.2, 13.0], [5.1, 3.3, 1.7, 0.5, 21.0], [5.1, 3.8, 1.5, 0.3, 29.0], [5.1, 3.5, 1.4, 0.3, 76.0], [5.2, 3.5, 1.5, 0.2, 47.0], [5.2, 2.7, 3.9, 1.4, 69.0], [5.2, 4.1, 1.5, 0.1, 84.0], [5.3, 3.7, 1.5, 0.2, 30.0], [5.4, 3.9, 1.7, 0.4, 1.0], [5.4, 3.4, 1.5, 0.4, 7.0], [5.4, 3.0, 4.5, 1.5, 26.0], [5.4, 3.7, 1.5, 0.2, 83.0], [5.4, 3.9, 1.3